In [1]:
using JLD

In [10]:
using Images

GROUND_TRUTH_FOLDER = "/Users/manvithaponnapati/RAISR/train_data/ground_truth/"
LOW_RES_FOLDER = "/Users/manvithaponnapati/RAISR/train_data/low_res/"
INTERPOLATED_FOLDER = "/Users/manvithaponnapati/RAISR/train_data/interpolated/"

"/Users/manvithaponnapati/RAISR/train_data/interpolated/"

In [11]:
scale_factor = 2

2

In [12]:
# Pkg.add("Distributions")
@everywhere function hashkey(patch,weights_matrix,patchsize)
    gx,gy = imgradients(patch,KernelFactors.ando3)
    gx = reshape(gx,patchsize*patchsize,1)
    gy = reshape(gy,patchsize*patchsize,1)
    GT = transpose([gx gy])
   
    GTWG = GT*weights_matrix*transpose(GT)
    
    eigen_max = eigmax(GTWG)
    eigen_min = eigmin(GTWG)
    eigen_vector_max = eigvecs(GTWG)[:,1]
    eigen_vector_min = eigvecs(GTWG)[:,2]
    gradient_angle = atan2(eigen_vector_max[2],eigen_vector_max[1])
    if gradient_angle < 0
        gradient_angle = gradient_angle + pi
    end
    lamda = abs(sqrt(complex(eigen_max)))/Qstrength
    u =(sqrt(complex(eigen_max)) -  sqrt(complex(eigen_min)))/(sqrt(complex(eigen_max)) +  sqrt(complex(eigen_min)))/Qcoherence
    angle = floor(gradient_angle/pi*Qangle)

    if lamda < 0.0001
        strength = 1
    elseif lamda > 0.001
        strength = 3
    else
        strength = 2
    end

    u = abs(u)
    if u < 0.25
        coherence = 1
    elseif u > 0.5
        coherence = 3
    else
        coherence = 2
    end


    # Bound the output to the desired ranges
    if angle > 23
        angle = 23
    elseif angle <= 0
        angle = 1
    end
    angle = Int(angle)
    return angle,strength,coherence
end

In [13]:
using Distributions
#Calculate image properties
#Taking 3x3 patches on the imahes 
Qangle = 24
Qstrength = 3
Qcoherence = 3
patchsize = 11
gradientsize = 9
Q = zeros((Qangle, Qstrength, Qcoherence, scale_factor*scale_factor, patchsize*patchsize, patchsize*patchsize))
V = zeros((Qangle, Qstrength, Qcoherence, scale_factor*scale_factor, patchsize*patchsize))
mark = zeros((Qstrength*Qcoherence, Qangle, scale_factor*scale_factor))

start_index = Int(ceil(patchsize/2))
weights_matrix = rand(Normal(0, 2), 9*9)
weights_matrix = Array(Diagonal(weights_matrix))
#the really really twosted conversions on JuliaImages
filenames = readdir(INTERPOLATED_FOLDER)
for file in filenames
     if (contains("$file", ".jpg")) == true
        println("File nm: $file")
        low_res_im = load(INTERPOLATED_FOLDER*file)
        ground_truth_im = load(GROUND_TRUTH_FOLDER*file)
        
        colorview_rgb = channelview(YCbCr.(low_res_im))
        color_ve = colorview_rgb[1,:,:]/255
        
        colorview_rgb_gt = channelview(YCbCr.(ground_truth_im))
        color_ve_gt = colorview_rgb_gt[1,:,:]/255
        
        w,h = size(color_ve)
        println("Starting file")
        @time for row in start_index:Int(w)-start_index
            for col in start_index:Int(h)-start_index
                
                patch = color_ve[row-5:row+5,col-5:col+5]
                gradientpatch = color_ve[row-4:row+4,col-4:col+4]
                angle,strength,coherence = hashkey(gradientpatch,weights_matrix,9)
#                 println("$angle,$strength,$coherence")
                # Get pixel type
                pixeltype = ((row-start_index) % scale_factor) * scale_factor + ((col-start_index) % scale_factor)
                pixelHR = color_ve_gt[row,col]
#                 println(pixelHR)
                # Compute A'A and A'b
                
                wp,hp = size(patch)
                patch_1 = reshape(patch,wp*hp,1)
                patch = reshape(patch,wp*hp)
                ATA = dot(transpose(patch_1),patch_1)
                ATb = patch*pixelHR
                
                Q[angle,strength,coherence,pixeltype+1,:,:] += ATA
                V[angle,strength,coherence,pixeltype+1,:] +=ATb
                mark[coherence*3+strength, angle, pixeltype+1] += 1
            end
        end

    end
end



File nm: 100007.jpg
Starting file
 75.528494 seconds (52.33 M allocations: 39.885 GiB, 7.64% gc time)
File nm: 100039.jpg
Starting file
 96.301738 seconds (52.34 M allocations: 39.885 GiB, 5.94% gc time)
File nm: 100075.jpg
Starting file
 84.521073 seconds (52.34 M allocations: 39.885 GiB, 6.38% gc time)
File nm: 100080.jpg
Starting file
 73.597811 seconds (52.33 M allocations: 39.885 GiB, 7.14% gc time)
File nm: 100098.jpg
Starting file
 73.536298 seconds (52.33 M allocations: 39.885 GiB, 7.63% gc time)
File nm: 100099.jpg
Starting file
 83.343010 seconds (52.34 M allocations: 39.885 GiB, 6.71% gc time)
File nm: 10081.jpg
Starting file
 78.568530 seconds (52.30 M allocations: 39.884 GiB, 7.12% gc time)
File nm: 101027.jpg
Starting file
 80.976432 seconds (52.34 M allocations: 39.885 GiB, 7.08% gc time)
File nm: 101084.jpg
Starting file
 87.373191 seconds (52.35 M allocations: 39.885 GiB, 6.67% gc time)
File nm: 102062.jpg
Starting file
 96.870657 seconds (52.34 M allocations: 39.885 G

 81.082093 seconds (52.27 M allocations: 39.883 GiB, 7.09% gc time)
File nm: 147062.jpg
Starting file
 88.339125 seconds (52.33 M allocations: 39.885 GiB, 6.87% gc time)
File nm: 147077.jpg
Starting file
 78.371116 seconds (52.34 M allocations: 39.885 GiB, 6.94% gc time)
File nm: 147080.jpg
Starting file
 72.886925 seconds (52.33 M allocations: 39.885 GiB, 7.90% gc time)
File nm: 15004.jpg
Starting file
 69.446670 seconds (52.34 M allocations: 39.885 GiB, 8.42% gc time)
File nm: 15011.jpg
Starting file
 73.605073 seconds (52.34 M allocations: 39.885 GiB, 7.68% gc time)
File nm: 15062.jpg
Starting file
 75.170989 seconds (52.34 M allocations: 39.885 GiB, 8.11% gc time)
File nm: 15088.jpg
Starting file
 70.521624 seconds (52.32 M allocations: 39.884 GiB, 8.02% gc time)
File nm: 151087.jpg
Starting file
 72.146438 seconds (52.32 M allocations: 39.884 GiB, 8.21% gc time)
File nm: 153077.jpg
Starting file
 83.911938 seconds (52.31 M allocations: 39.884 GiB, 6.90% gc time)
File nm: 153093.jp

 68.074391 seconds (52.33 M allocations: 39.885 GiB, 9.19% gc time)
File nm: 198087.jpg
Starting file
 68.044638 seconds (52.32 M allocations: 39.884 GiB, 9.37% gc time)
File nm: 20008.jpg
Starting file
 68.405383 seconds (52.33 M allocations: 39.885 GiB, 9.31% gc time)
File nm: 20069.jpg
Starting file
 66.778595 seconds (52.33 M allocations: 39.885 GiB, 9.52% gc time)
File nm: 201080.jpg
Starting file
 69.399565 seconds (52.28 M allocations: 39.884 GiB, 9.48% gc time)
File nm: 2018.jpg
Starting file
 68.879957 seconds (52.32 M allocations: 39.885 GiB, 9.54% gc time)
File nm: 202000.jpg
Starting file
 68.364540 seconds (52.32 M allocations: 39.885 GiB, 9.11% gc time)
File nm: 202012.jpg
Starting file
 68.057050 seconds (52.34 M allocations: 39.885 GiB, 9.42% gc time)
File nm: 206062.jpg
Starting file
 69.247752 seconds (52.34 M allocations: 39.885 GiB, 9.56% gc time)
File nm: 206097.jpg
Starting file
 67.620923 seconds (52.33 M allocations: 39.885 GiB, 9.75% gc time)
File nm: 207038.jp

 71.903386 seconds (52.35 M allocations: 39.885 GiB, 8.42% gc time)
File nm: 268074.jpg
Starting file
 74.787152 seconds (52.35 M allocations: 39.885 GiB, 8.05% gc time)
File nm: 27059.jpg
Starting file
 75.141947 seconds (52.34 M allocations: 39.885 GiB, 7.76% gc time)
File nm: 271008.jpg
Starting file
 79.030384 seconds (52.34 M allocations: 39.885 GiB, 7.26% gc time)
File nm: 271031.jpg
Starting file
 76.455254 seconds (52.33 M allocations: 39.885 GiB, 8.07% gc time)
File nm: 274007.jpg
Starting file
 82.604695 seconds (52.34 M allocations: 39.885 GiB, 6.76% gc time)
File nm: 277053.jpg
Starting file
 93.926549 seconds (52.34 M allocations: 39.885 GiB, 5.98% gc time)
File nm: 277095.jpg
Starting file
 77.680342 seconds (52.33 M allocations: 39.885 GiB, 6.69% gc time)
File nm: 279005.jpg
Starting file
 73.108645 seconds (52.31 M allocations: 39.884 GiB, 7.61% gc time)
File nm: 28075.jpg
Starting file
 73.372912 seconds (52.35 M allocations: 39.885 GiB, 7.45% gc time)
File nm: 28083.j

 80.943207 seconds (52.33 M allocations: 39.885 GiB, 7.37% gc time)
File nm: 388067.jpg
Starting file
 81.837683 seconds (52.34 M allocations: 39.885 GiB, 7.01% gc time)
File nm: 393035.jpg
Starting file
 82.168372 seconds (52.18 M allocations: 39.881 GiB, 6.89% gc time)
File nm: 41004.jpg
Starting file
 83.828549 seconds (52.33 M allocations: 39.885 GiB, 6.40% gc time)
File nm: 41006.jpg
Starting file
 90.337773 seconds (52.34 M allocations: 39.885 GiB, 6.19% gc time)
File nm: 41025.jpg
Starting file
 85.691035 seconds (52.32 M allocations: 39.884 GiB, 6.39% gc time)
File nm: 41029.jpg
Starting file
 98.634568 seconds (52.34 M allocations: 39.885 GiB, 5.68% gc time)
File nm: 41085.jpg
Starting file
103.103009 seconds (52.34 M allocations: 39.885 GiB, 5.71% gc time)
File nm: 41096.jpg
Starting file
 86.948336 seconds (52.33 M allocations: 39.885 GiB, 6.21% gc time)
File nm: 42044.jpg
Starting file
112.918093 seconds (52.23 M allocations: 39.882 GiB, 5.64% gc time)
File nm: 42078.jpg
St

In [19]:
P = zeros((patchsize*patchsize, patchsize*patchsize, 7))
rotate = zeros((patchsize*patchsize, patchsize*patchsize))
flip = zeros((patchsize*patchsize, patchsize*patchsize))

for i in range(1, patchsize*patchsize-1)
    i1 = i % patchsize
    i2 = Int(ceil(i / patchsize))
    j = (patchsize * patchsize - patchsize + i2 - patchsize * i1)+1 
    rotate[j,i] = 1
    k = patchsize * (i2 + 1) - i1 - 4
    flip[k,i] = 1
end

for i in range(1, 7)
    i1 = i % 4
    i2 = ceil(i / 4)
    P[:,:,i] = dot((flip^i2),(rotate^i1))
end
Qextended = zeros((Qangle, Qstrength, Qcoherence, scale_factor*scale_factor, patchsize*patchsize, patchsize*patchsize))
Vextended = zeros((Qangle, Qstrength, Qcoherence, scale_factor*scale_factor, patchsize*patchsize))

for pixeltype in range(1, scale_factor*scale_factor)
    for angle in range(1, Qangle)
        for strength in range(1, Qstrength)
            for coherence in range(1, Qcoherence)
                for m in range(1, 7)
                    m1 = m % 4
                    m2 = floor(m / 4)
                    newangleslot = angle
                    if m2 == 1
                        newangleslot = Qangle-angle-1
                    end
                    newangleslot = Int(newangleslot-Qangle/2*m1)
                    while newangleslot < 0
                        newangleslot += Qangle
                    end
                    
                    newQ = transpose(P[:,:,m])*(Q[angle,strength,coherence,pixeltype,:,:])*(P[:,:,m])
                    newV = transpose(P[:,:,m])*(V[angle,strength,coherence,pixeltype,:])
                    Qextended[newangleslot+1,strength,coherence,pixeltype,:,:] += newQ
                    Vextended[newangleslot+1,strength,coherence,pixeltype,:] += newV
                end
            end
        end
    end
end
Q += Qextended
V += Vextended

LoadError: [91mBoundsError: attempt to access 121×121 Array{Float64,2} at index [127, 111][39m

In [17]:
using Krylov
h = zeros((Qangle, Qstrength, Qcoherence, scale_factor*scale_factor, patchsize*patchsize))
operationcount = 0
totaloperations = scale_factor * scale_factor * Qangle * Qstrength * Qcoherence
for pixeltype in range(1, scale_factor*scale_factor)
    for angle in range(1, Qangle)
        for strength in range(1, Qstrength)
            for coherence in range(1, Qcoherence)
                println(" K: index $pixeltype,$angle, $strength, $coherence")
                operationcount += 1
                x,stats = cgls(Q[angle,strength,coherence,pixeltype,:,:], V[angle,strength,coherence,pixeltype,:])
                h[angle,strength,coherence,pixeltype,:] = x
            end
        end
    end
end

 K: index 1,1, 1, 1
 K: index 1,1, 1, 2
 K: index 1,1, 1, 3
 K: index 1,1, 2, 1
 K: index 1,1, 2, 2
 K: index 1,1, 2, 3
 K: index 1,1, 3, 1
 K: index 1,1, 3, 2
 K: index 1,1, 3, 3
 K: index 1,2, 1, 1
 K: index 1,2, 1, 2
 K: index 1,2, 1, 3
 K: index 1,2, 2, 1
 K: index 1,2, 2, 2
 K: index 1,2, 2, 3
 K: index 1,2, 3, 1
 K: index 1,2, 3, 2
 K: index 1,2, 3, 3
 K: index 1,3, 1, 1
 K: index 1,3, 1, 2
 K: index 1,3, 1, 3
 K: index 1,3, 2, 1
 K: index 1,3, 2, 2
 K: index 1,3, 2, 3
 K: index 1,3, 3, 1
 K: index 1,3, 3, 2
 K: index 1,3, 3, 3
 K: index 1,4, 1, 1
 K: index 1,4, 1, 2
 K: index 1,4, 1, 3
 K: index 1,4, 2, 1
 K: index 1,4, 2, 2
 K: index 1,4, 2, 3
 K: index 1,4, 3, 1
 K: index 1,4, 3, 2
 K: index 1,4, 3, 3
 K: index 1,5, 1, 1
 K: index 1,5, 1, 2
 K: index 1,5, 1, 3
 K: index 1,5, 2, 1
 K: index 1,5, 2, 2
 K: index 1,5, 2, 3
 K: index 1,5, 3, 1
 K: index 1,5, 3, 2
 K: index 1,5, 3, 3
 K: index 1,6, 1, 1
 K: index 1,6, 1, 2
 K: index 1,6, 1, 3
 K: index 1,6, 2, 1
 K: index 1,6, 2, 2


 K: index 2,23, 3, 2
 K: index 2,23, 3, 3
 K: index 2,24, 1, 1
 K: index 2,24, 1, 2
 K: index 2,24, 1, 3
 K: index 2,24, 2, 1
 K: index 2,24, 2, 2
 K: index 2,24, 2, 3
 K: index 2,24, 3, 1
 K: index 2,24, 3, 2
 K: index 2,24, 3, 3
 K: index 3,1, 1, 1
 K: index 3,1, 1, 2
 K: index 3,1, 1, 3
 K: index 3,1, 2, 1
 K: index 3,1, 2, 2
 K: index 3,1, 2, 3
 K: index 3,1, 3, 1
 K: index 3,1, 3, 2
 K: index 3,1, 3, 3
 K: index 3,2, 1, 1
 K: index 3,2, 1, 2
 K: index 3,2, 1, 3
 K: index 3,2, 2, 1
 K: index 3,2, 2, 2
 K: index 3,2, 2, 3
 K: index 3,2, 3, 1
 K: index 3,2, 3, 2
 K: index 3,2, 3, 3
 K: index 3,3, 1, 1
 K: index 3,3, 1, 2
 K: index 3,3, 1, 3
 K: index 3,3, 2, 1
 K: index 3,3, 2, 2
 K: index 3,3, 2, 3
 K: index 3,3, 3, 1
 K: index 3,3, 3, 2
 K: index 3,3, 3, 3
 K: index 3,4, 1, 1
 K: index 3,4, 1, 2
 K: index 3,4, 1, 3
 K: index 3,4, 2, 1
 K: index 3,4, 2, 2
 K: index 3,4, 2, 3
 K: index 3,4, 3, 1
 K: index 3,4, 3, 2
 K: index 3,4, 3, 3
 K: index 3,5, 1, 1
 K: index 3,5, 1, 2
 K: index

In [18]:
save("learned_filters_full_119.jld","filters",h,"weights",weights_matrix)
learned_filters = load("learned_filters_full_119.jld","filters")

24×3×3×4×121 Array{Float64,5}:
[:, :, 1, 1, 1] =
 6.82947e-5  6.83561e-5  6.71144e-5
 6.83087e-5  6.83394e-5  6.71871e-5
 6.82765e-5  6.83325e-5  6.71442e-5
 6.8279e-5   6.83324e-5  6.71154e-5
 6.83286e-5  6.83325e-5  6.70261e-5
 6.83767e-5  6.83375e-5  6.69985e-5
 6.81974e-5  6.83491e-5  6.68914e-5
 6.83515e-5  6.83299e-5  6.70734e-5
 6.82595e-5  6.83257e-5  6.70607e-5
 6.82712e-5  6.83353e-5  6.6955e-5 
 6.83367e-5  6.8334e-5   6.70273e-5
 6.82968e-5  6.83187e-5  6.7085e-5 
 6.8238e-5   6.83307e-5  6.71693e-5
 6.82696e-5  6.83558e-5  6.71745e-5
 6.83459e-5  6.8342e-5   6.72747e-5
 6.82913e-5  6.83309e-5  6.72236e-5
 6.82909e-5  6.83508e-5  6.70342e-5
 6.82915e-5  6.83362e-5  6.70348e-5
 6.82834e-5  6.83363e-5  6.70683e-5
 6.83063e-5  6.83466e-5  6.70688e-5
 6.83387e-5  6.83422e-5  6.7122e-5 
 6.82808e-5  6.83652e-5  6.70639e-5
 6.8275e-5   6.83643e-5  6.70607e-5
 0.0         0.0         0.0       

[:, :, 2, 1, 1] =
 6.83029e-5  6.83078e-5  6.70621e-5
 6.83454e-5  6.83094e-5  6.72572